# Classical Monte Carlo of $S_N$ Model on Hypertree Lattice

## Statistical Mechanics

### Generic Model
The most general StatMech model we consider is specified by the following ingredients:
* **Lattice**: we will only consider *bipartite* lattice, on which the block Gibbs sampling can be carried out.
 * The bipartite lattice contains two **sublattices** labeled by *A* and *B*.
 * The **boundary**, labeled by *C*, refers to the sites that will not be updated in Monte Carlo. The boundary configuration is fixed by the boundary condition and will not be updated. 
* **Group**: on each site, the degree of freedom is a permutation group element $g_i\in S_n$, which we will called the "spin" hereinafter.
 * Each spin state will be represented as a vector in the canonical representation of $S_n$. For example, for $S_2$ group: $|()\rangle=(1,0)^\intercal$, $|(12)\rangle=(0,1)^\intercal.$
* **Hamiltonian**:
$$H[g] = -\sum_{\langle ij\rangle} K_{ij}\chi(g_i,g_j),$$
where $\chi(g_i,g_j)=\text{Tr}g_i^{-1}g_j$ is the group character (the number of cycles). The partition function is given by
$$Z = -\sum_{[g|A\cup B]}e^{-H[g]} = e^{-F[g|C]}.$$
With given boundary, the free energy $F$ is a function of the boundary configuration.

### Monte Carlo Algorithm
The basic idea of importance sampling is to set up a suitable Markov chain that draws configurations according to the Boltzmann weight
$$P[g] = \frac{e^{-H[g]}}{Z}.$$
There are two classes of update schemes:
* Local update algorithms (Metropolis, **Heat-Bath**)
* Cluster update algorithms (Swendsen-Wang Cluster, Wolff Cluster)

The cluster update greately reduces the dynamic scaling at the critical point, which speed up the convergence in the critical region. Cluster update works well for Potts model, where the spins interacts via delta function: such that the domain wall between all different spins are of the same tension (energy). In the Wolff cluster algorithm, a site is first choosen, and neighboring sites of the same spin can attach to the cluster with probability $p$ (sites of different spins should not attach), and finally the whole cluster is flipped. The transition rate can satisfy the detail balance because the intereial weight is the same ($=p^{C-1}$ where $C$ is number of cluster site, this is because the cluster is grown following a tree, and all trees are equivalent in bond number if the site number is fixed), so the only difference is on the boundary. There are two types of cluster boundaries: boudaries between different spins (of tension 0) and boundary between same spins (of tension $-\ln p$). After flipping, the two types of boundary will switch their positions, so it can match the detail balance condition. More precisely, the partition function for $n$-state Potts model can be cast into
$$Z\propto\sum_{\text{config}}p^{\text{#link}}n^{\text{#cluster}}.$$
This is due to the nice delta function structure. However for our $S_n$ model, the delta interaction is soften in some sense. Then the partition function no longer has the nice link and cluster structure, which make it hard to do cluster update.

While it remains a question of <font color='red'>how to design a cluster update algorithm for $S_n$ model</font>, we will try the local update first, in particular the heat-bath update. The current code uses **block Gibbs sampling** + **heat-bath update**, alternatively sampling on A and B sublattices.

Question: <font color='red'>How to determine equilibrium?</font>

### Temperature Profiling

Question: <font color='red'>How to calculate free energy?</font>

## Code Structure
Objects:
* **Group**
* **Lattice** (HypertreeLattice)
* **Model** (LatticeModel)
    * Wrapper of the FORTRAN extension MC (provides interface for data transfer and control).

In [1]:
%reload_ext snakeviz
%run 'MC.py'

### <code>Group</code> Object

<code>group = Group(n)</code> represents a permutation group $S_n$.
* Each element is a tuple of $n$ numbers, representing the permutation of $(0,1,...,n-1)$.
* <code>group.element</code> gives the list of all group elements.
* <code>group.dof</code> gives the order of the group, i.e. $n!$, which is also the degrees of freedom on each site.
* <code>group.index</code> is a dictionary that maps group element to its index.

In [20]:
'''multiplication table'''
group = Group(3)
[[group.index[group.multiply(g,h)] for h in group] for g in group]

[[0, 1, 2, 3, 4, 5],
 [1, 0, 4, 5, 2, 3],
 [2, 3, 0, 1, 5, 4],
 [3, 2, 5, 4, 0, 1],
 [4, 5, 1, 0, 3, 2],
 [5, 4, 3, 2, 1, 0]]

In [23]:
'''characters'''
{g: group.character(g) for g in group}

{(0, 1, 2): 3,
 (0, 2, 1): 2,
 (1, 0, 2): 2,
 (1, 2, 0): 1,
 (2, 0, 1): 1,
 (2, 1, 0): 2}

The character of a permutation is defined as the number of cycles in the permutation, denoted as $\ln\text{Tr}g$, ranging from $1$ to $n$. Upon initialization, the <code>Group</code> object generates the table of
$$\chi(g_1,g_2)=\ln\text{Tr} g_1 g_2^{-1}-n.$$
One can show that $\chi(g_1,g_2)=\chi(g_2,g_1)$ and $\forall g: \chi(gg_1,gg_2)=\chi(g_1g,g_2g)=\chi(g_1,g_2)$.
* This is the interaction function between two spins across a bond.
* The shift $-n$ is needed to avoid partition weight overflow at low temperature. 

In [24]:
Group(3).chi

[[0, -1, -1, -2, -2, -1],
 [-1, 0, -2, -1, -1, -2],
 [-1, -2, 0, -1, -1, -2],
 [-2, -1, -1, 0, -2, -1],
 [-2, -1, -1, -2, 0, -1],
 [-1, -2, -2, -1, -1, 0]]

### <code>Lattice</code> Object

Site arranging rules: AB must before C, because will not allocate for C, contineous space for AB, within each sublattice dont scatter around.

In [2]:
HypertreeLattice(4)

[<2,0,0>, <2,1,0>, <2,2,0>, <2,3,0>, <1,0,0>, <1,0,1>, <1,1,0>, <1,1,1>, <0,0,0>, <0,0,1>, <0,0,2>, <0,0,3>]

### <code>Model</code> Object

The CPU intensive Monte Carlo samplings are performed by the FORTRAN extension MC. The module MC contains two parts:
* core (for MC sampling and update macroscopic observibles)
* physics (for data collection and measurements)

In [3]:
import MC
print(MC.__doc__)

This module 'MC' is auto-generated with f2py (version:2).
Functions:
Fortran 90/95 modules:
  core --- chi,jlst,klst,nsite,na,dof,nlst,config,nc,irng,energy,nb,hist,init(),run(),get_energy(),get_hist(),dump(),load()  physics --- nspin,spins,energy2,magnet1,monitor,energy1,magnet2,measure().


## Hypertree Model

### Example
Generate model, run, get hist and energy, make measurements.
* use **block Gibbs sampling** + **heatbath update**, alternatively sampling on A and B sublattices.
 * A sublattice is the one that contains the most UV layer
 * B sublattice contains the rest
 * the boundary is labeled by C
* energy is calculated for all sites including the boundary spins.
* hist and magnetization only conts the bulk spins, boundary spins are not taken into account.

In [2]:
sys = LatticeModel(HypertreeLattice(4,[1.,0.2]),Group(3)).run(100)
for k in range(10):
    sys.run(mode = 1)
    print(sys.config,sys.hist,sys.energy)
sys.measure(1000,monitor=[0,1,2,3])

[0 2 0 1 0 0 5 1 0 0 0 0] [4 2 1 0 0 1] 5.199999999999999
[0 3 3 3 0 5 1 3 0 0 0 0] [2 1 0 4 0 1] 7.599999999999999
[4 1 3 1 1 0 2 1 0 0 0 0] [1 4 1 1 1 0] 9.999999999999998
[0 2 0 3 0 0 2 0 0 0 0 0] [5 0 2 1 0 0] 3.9999999999999987
[2 0 0 3 0 5 2 2 0 0 0 0] [3 0 3 1 0 1] 9.999999999999996
[0 5 2 2 0 0 3 2 0 0 0 0] [3 0 3 1 0 1] 4.399999999999997
[0 3 2 3 0 0 2 2 0 0 0 0] [3 0 3 2 0 0] 3.5999999999999974
[1 2 2 4 0 0 0 0 0 0 0 0] [4 1 2 0 1 0] 6.799999999999999
[1 2 0 1 4 0 0 0 0 0 0 0] [4 2 1 0 1 0] 8.399999999999999
[0 5 0 5 0 1 5 0 0 0 0 0] [4 1 0 0 0 3] 6.799999999999997


{'energy1': 6.492800000000001,
 'energy2': 47.164479999999934,
 'magnet1': array([ 0.416   ,  0.12875 ,  0.141375,  0.08725 ,  0.0875  ,  0.139125]),
 'magnet2': array([[ 0.20721875,  0.04709375,  0.05221875,  0.03079688,  0.0299375 ,
          0.04873438],
        [ 0.04709375,  0.0350625 ,  0.01214063,  0.00970312,  0.009875  ,
          0.014875  ],
        [ 0.05221875,  0.01214063,  0.04076562,  0.01082812,  0.01059375,
          0.01482812],
        [ 0.03079688,  0.00970312,  0.01082812,  0.01996875,  0.0056875 ,
          0.01026563],
        [ 0.0299375 ,  0.009875  ,  0.01059375,  0.0056875 ,  0.02109375,
          0.0103125 ],
        [ 0.04873438,  0.014875  ,  0.01482812,  0.01026563,  0.0103125 ,
          0.04010938]]),
 'spins': array([[ 0.578,  0.252,  0.234,  0.166],
        [ 0.092,  0.162,  0.152,  0.172],
        [ 0.124,  0.167,  0.179,  0.172],
        [ 0.033,  0.127,  0.14 ,  0.162],
        [ 0.037,  0.118,  0.136,  0.161],
        [ 0.136,  0.174,  0.159,  0.

### Profiling

In [1]:
%reload_ext snakeviz
%run 'MC.py'
%snakeviz LatticeModel(HypertreeLattice(4,[1.,0.2]),Group(3)).run(10000,0)

 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmp4fxjdkbk'. 
